# 空氣污染監測網 網路爬蟲實作練習


* 能夠利用 selenium + BeautifulSoup 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料





### ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料

In [1]:
'''
Your Code
'''

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import time

browser = webdriver.Chrome(executable_path='chromedriver')
browser.implicitly_wait(5)
browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value('11')
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2018')
browser.find_element_by_id('ctl05_btnQuery').click()

time.sleep(3)

html_source = browser.page_source
soup = BeautifulSoup(html_source, 'html.parser')
table = soup.find('table', class_='TABLE_G')

d = {}
for ele in table.find_all('tr')[1:]:
    try:
        if len(ele.find_all('td')) == 5:
            detection_name = ele.find_all('td')[0].text
            detection_date = ele.find_all('td')[2].text
            detection_value = float(ele.find_all('td')[3].text)
        else:
            detection_date = ele.find_all('td')[0].text
            detection_value = float(ele.find_all('td')[1].text)
        d.setdefault(detection_name, {})[detection_date] = detection_value
    except:
        d[detection_name] = {}
    
df = pd.DataFrame(d)
df = df.reset_index()
df[df['index'] <= '2018/08'][['index','SO2']]

,index,SO2
0,2018/01,1.8
1,2018/02,1.9
2,2018/03,2.2
3,2018/04,2.3
4,2018/05,3.1
5,2018/06,2.7
6,2018/07,2.2
7,2018/08,2.4


### ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [2]:
'''
Your Code
'''
df[df['index'] <= '2018/08'][['index', 'SO2', 'CO']]

,index,SO2,CO
0,2018/01,1.8,0.34
1,2018/02,1.9,0.44
2,2018/03,2.2,0.40
3,2018/04,2.3,0.38
4,2018/05,3.1,0.34
5,2018/06,2.7,0.29
6,2018/07,2.2,0.21
7,2018/08,2.4,0.30


In [3]:
browser.close()